<a href="https://colab.research.google.com/github/andreacini/ml-19-20/blob/master/08_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Machine Learning

Prof. Cesare Alippi  
Andrea Cini ([`andrea.cini@usi.ch`](mailto:andrea.cini@usi.ch) )    
Daniele Zambon ([`daniele.zambon@usi.ch`](mailto:daniele.zambon@usi.ch)  )

---
# Lab 08: Forecasting


## A simple problem

$$x(t) = \sin(\omega \,t) + \eta, \qquad \eta\sim N(0,\sigma^2)$$

In [0]:
import numpy as np
import matplotlib.pyplot as plt

omega = 30
t = np.linspace(0, 2 * np.pi, 3000)
x = np.sin(omega * t)

print("x", x.shape)
plt.hist(x, density=True);

Time matters!

In [0]:
plt.figure(figsize=(12, 5))
plt.plot(x, '.')

### Let's divide data into windows

```
            historical data                   | future observations
--------------------------------------------------------------------
x(1) x(2) ... x(t-p) x(t-p+1) ... x(t-1) x(t) | x(t+1)  x(t+2) ...
                    \________________________/| \____/
                       time window            | value to 
                                              | be predicted
```

In [0]:
# Time windows
def get_time_windows(sequence, window_size, train_size):
    time = np.arange(sequence.shape[0])
    xseq = []
    yseq = []
    for t in time[:-window_size]:
        xseq.append(sequence[t:t+window_size])
        yseq.append(sequence[t+window_size])
    xseq = np.array(xseq)
    yseq = np.array(yseq)
    #train-test split
    t = int(xseq.shape[0] * train_size)
    return xseq[:t], xseq[t:], yseq[:t], yseq[t:]


In [0]:
# Prepare the data
p= 5

x_train, x_test, y_train, y_test = get_time_windows(sequence=x, window_size=p, train_size=0.7)
print("training data", x_train.shape, y_train.shape)
print("test data", x_test.shape, y_test.shape)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Build a model 
def get_model(win_size):
    model = Sequential()
    num_neurons = win_size*2
    model.add(Dense(num_neurons, input_shape=(win_size,), activation="tanh"))
    model.add(Dense(1, activation="linear"))

    model.compile(optimizer="adam", loss="mse")
    return model

In [0]:
p= 5

# Prepare the data
x_train, x_test, y_train, y_test = get_time_windows(sequence=x, window_size=p, train_size=0.7)

# Train a model
model = get_model(p)
model.fit(x_train, y_train, epochs=100)

In [0]:
# Predictions
y_pred = model.predict(x_test)

plt.figure(figsize=(12, 5))
plt.plot(np.arange(y_test.shape[0]), y_test, '.', label="ground truth")
plt.plot(np.arange(y_test.shape[0]), y_pred, '.', label="predicted")
plt.legend()

In [0]:
# Predict on our prediction
def predict_rec(n, model, x_init):
    p = x_init.shape[0]

    y_new = np.zeros(p+n)
    y_new[:p] = x_init.copy()

    for j in range(n):
        y_tmp = model.predict(y_new[j:j+p][None, ...])
        y_new[j+p] = y_tmp[-1,0]
    return y_new[p:]

In [0]:
# one-step prediction
y_pred = model.predict(x_test)

# one-step prediction
n_last = 250
y_new = predict_rec(n=n_last, model=model, x_init=x_test[-n_last-1])

plt.figure(figsize=(12, 5))
plt.plot(np.arange(y_test.shape[0]), y_test, '.', label="ground truth")
plt.plot(np.arange(y_test.shape[0]), y_pred, '.', label="predicted")
plt.plot(np.arange(y_test.shape[0]-n_last, y_test.shape[0]), y_new, '.', label="new")
plt.legend()

### A slightly more complex problem

In [0]:
t = np.linspace(0, 2 * np.pi, 2000)
x = np.sin(omega * t)*np.sin(1.74*omega * t)  + np.random.normal(loc=0, scale=.2, size=t.shape)

plt.figure(figsize=(12, 5))
plt.plot(x, '-')

In [0]:
p=5

# Prepare the data
x_train, x_test, y_train, y_test = get_time_windows(sequence=x, window_size=p, train_size=0.7)

# Train a model
model = get_model(p)
model.fit(x_train, y_train, epochs=100)

In [0]:
# one-step prediction
y_pred = model.predict(x_test)

# recursive prediction
n_last = 150
y_new = predict_rec(n=n_last, model=model, x_init=x_test[-n_last-1])

plt.figure(figsize=(12, 5))
plt.plot(np.arange(y_test.shape[0]), y_test, '-', label="ground truth")
plt.plot(np.arange(y_test.shape[0]), y_pred, '-', label="predicted")
plt.plot(np.arange(y_test.shape[0]-n_last, y_test.shape[0]), y_new, '.', label="new")
plt.legend()